In [1]:
# Import modules
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from secret import username, password
from sqlalchemy import create_engine

#### Clean international revenue data set

In [2]:
master_df = pd.read_csv("Master_Table.csv")

In [3]:
master_df.head(3)

,Rank,Title,Year,Lifetime Gross,GrossNumeric
0,1,Avengers: Endgame,2019,"$2,797,800,564",2797800564
1,2,Avatar,2009,"$2,790,439,000",2790439000
2,3,Titanic,1997,"$2,195,169,138",2195169138


In [4]:
len(master_df)

10200

In [5]:
# Getting the duplicate movie by title and year released. 
duplicated_master_df = master_df.loc[master_df.duplicated(subset=["Title","Year"]), "Title"].unique()
duplicated_master_df

array(['War of the Buttons', 'My Love'], dtype=object)

In [6]:
# Get all the data for the duplicate movie 
duplicated_master_df = master_df.loc[master_df["Title"] == "War of the Buttons"]
duplicated_master_df

,Rank,Title,Year,Lifetime Gross,GrossNumeric
5983,5984,War of the Buttons,2011,"$15,082,409",15082409
6643,6644,War of the Buttons,2011,"$12,326,773",12326773


In [7]:
# Get all the data for the duplicate movie 
duplicated_master_df = master_df.loc[master_df["Title"] == "My Love"]
duplicated_master_df

,Rank,Title,Year,Lifetime Gross,GrossNumeric
5334,5335,My Love,2007,"$18,525,631",18525631
8994,8995,My Love,2007,"$6,660,413",6660413


In [8]:
# reset index
master_df.reset_index()

,index,Rank,Title,Year,Lifetime Gross,GrossNumeric
0,0,1,Avengers: Endgame,2019,"$2,797,800,564",2797800564
1,1,2,Avatar,2009,"$2,790,439,000",2790439000
2,2,3,Titanic,1997,"$2,195,169,138",2195169138
3,3,4,Star Wars: Episode VII - The Force Awakens,2015,"$2,068,223,624",2068223624
4,4,5,Avengers: Infinity War,2018,"$2,048,359,754",2048359754
...,...,...,...,...,...,...
10195,10195,10196,Swiss Army Man,2016,"$4,935,501",4935501
10196,10196,10197,Mes héros,2012,"$4,934,927",4934927
10197,10197,10198,Nymphomaniac: Vol. II,2014,"$4,934,725",4934725
10198,10198,10199,The Hairdresser,2010,"$4,934,394",4934394


In [9]:
# Drop row by index
clean_df = master_df.drop(index=master_df.index[[6643, 8994]])

In [10]:
len(clean_df)

10198

In [11]:
# Select columns
clean_df = clean_df[['Rank', 'Title', 'Year', 'GrossNumeric']]

In [12]:
clean_df.head()

,Rank,Title,Year,GrossNumeric
0,1,Avengers: Endgame,2019,2797800564
1,2,Avatar,2009,2790439000
2,3,Titanic,1997,2195169138
3,4,Star Wars: Episode VII - The Force Awakens,2015,2068223624
4,5,Avengers: Infinity War,2018,2048359754


In [13]:
# Rename columns
international_v2 = clean_df.rename(columns={'Rank': 'rank', 'Title': 'name_international', 'Year': 'year', 'GrossNumeric': 'international_revenue_usd'})

In [14]:
international_v2.head()

,rank,name_international,year,international_revenue_usd
0,1,Avengers: Endgame,2019,2797800564
1,2,Avatar,2009,2790439000
2,3,Titanic,1997,2195169138
3,4,Star Wars: Episode VII - The Force Awakens,2015,2068223624
4,5,Avengers: Infinity War,2018,2048359754


In [15]:
international_v2.set_index('rank', inplace=True)

In [16]:
international_v2.head()

,name_international,year,international_revenue_usd
rank,,,
1,Avengers: Endgame,2019,2797800564
2,Avatar,2009,2790439000
3,Titanic,1997,2195169138
4,Star Wars: Episode VII - The Force Awakens,2015,2068223624
5,Avengers: Infinity War,2018,2048359754


In [17]:
# Change name column to lower case to easier join 
international_v2['name_international'] = international_v2['name_international'].str.capitalize()

In [18]:
international_v2.head()

,name_international,year,international_revenue_usd
rank,,,
1,Avengers: endgame,2019,2797800564
2,Avatar,2009,2790439000
3,Titanic,1997,2195169138
4,Star wars: episode vii - the force awakens,2015,2068223624
5,Avengers: infinity war,2018,2048359754


In [19]:
international_v2.to_csv('Resources/Master_table_v3.csv')

#### Clean domestic revenue data

In [20]:
domestic_df = pd.read_csv('Resources/datasets_movies_v3.csv')

In [21]:
domestic_df['name'] = domestic_df['name'].str.capitalize()

In [24]:
domestic_df = domestic_df.drop(columns=['year','country'])

In [25]:
domestic_df.head()

,budget,company,director,genre,gross,name,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes
2,15000000,Paramount Pictures,Tony Scott,Action,179800601,Top gun,PG,5/16/86,110,6.9,Tom Cruise,236909,Jim Cash
3,18500000,Twentieth Century Fox Film Corporation,James Cameron,Action,85160248,Aliens,R,7/18/86,137,8.4,Sigourney Weaver,540152,James Cameron
4,9000000,Walt Disney Pictures,Randal Kleiser,Adventure,18564613,Flight of the navigator,PG,8/1/86,90,6.9,Joey Cramer,36636,Mark H. Baker


In [26]:
# Getting the duplicate movie by title and year released. 
duplicated_domestic_df = domestic_df.loc[domestic_df.duplicated(subset=["name"]), "name"].unique()
duplicated_domestic_df

array(['Dream lover', 'Boiling point', "Nobody's fool", 'Heat',
       'It takes two', 'Rumpelstiltskin', 'Hamlet', 'Bulletproof',
       'The cure', 'Gladiator', 'Chocolat', 'Where the heart is',
       'Saving grace', 'The in crowd', 'Behind enemy lines', 'Out cold',
       'Pulse', 'Insomnia', 'Hero', 'Bad company', 'Crossroads',
       'The rookie', 'The hunted', 'It runs in the family',
       'The phantom of the opera', 'Man on fire', 'Taxi', 'Proof',
       'The proposition', 'Fever pitch', 'Man of the house', 'Invincible',
       'The guardian', 'Running scared', 'Fearless', 'Black sheep',
       'Hairspray', 'Sunshine', 'The hitcher', 'Flawless', 'Twilight',
       'The eye', 'Mother', 'The stepfather', 'The unborn',
       "Everybody's fine", 'Trust', 'Fair game', 'Death at a funeral',
       'Morning glory', 'The girl with the dragon tattoo', 'Priest',
       'The three musketeers', 'Jane eyre', 'Unknown', 'Trespass',
       'The avengers', 'Les misèrables', 'Anna karenina',

In [27]:
# Check duplicated movie
duplicated_domestic_df = domestic_df.loc[domestic_df["name"] == "Dream lover"]
duplicated_domestic_df

,budget,company,director,genre,gross,name,rating,released,runtime,score,star,votes,writer
192,0,Metro-Goldwyn-Mayer (MGM),Alan J. Pakula,Crime,502237,Dream lover,R,4/2/86,104,5.1,Kristy McNichol,405,Jon Boorstin
1652,0,PolyGram Filmed Entertainment,Nicholas Kazan,Drama,256264,Dream lover,R,5/6/94,103,6.2,James Spader,3327,Nicholas Kazan


In [28]:
# Drop duplicates and keep first
domestic_revenue_v2 = domestic_df.drop_duplicates(subset=['name'], keep='first')

In [29]:
domestic_revenue_v2

,budget,company,director,genre,gross,name,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes
2,15000000,Paramount Pictures,Tony Scott,Action,179800601,Top gun,PG,5/16/86,110,6.9,Tom Cruise,236909,Jim Cash
3,18500000,Twentieth Century Fox Film Corporation,James Cameron,Action,85160248,Aliens,R,7/18/86,137,8.4,Sigourney Weaver,540152,James Cameron
4,9000000,Walt Disney Pictures,Randal Kleiser,Adventure,18564613,Flight of the navigator,PG,8/1/86,90,6.9,Joey Cramer,36636,Mark H. Baker
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6815,0,Fox Searchlight Pictures,Mandie Fletcher,Comedy,4750497,Absolutely fabulous: the movie,R,7/22/16,91,5.4,Jennifer Saunders,9161,Jennifer Saunders
6816,0,Siempre Viva Productions,Paul Duddridge,Drama,28368,Mothers and daughters,PG-13,5/6/16,90,4.9,Selma Blair,1959,Paige Cameron
6817,3500000,Warner Bros. Animation,Sam Liu,Animation,3775000,Batman: the killing joke,R,7/25/16,76,6.5,Kevin Conroy,36333,Brian Azzarello
6818,0,Borderline Presents,Nicolas Pesce,Drama,25981,The eyes of my mother,R,12/2/16,76,6.2,Kika Magalh„es,6947,Nicolas Pesce


In [30]:
len(domestic_revenue_v2)

6731

In [31]:
duplicated_dom_df = domestic_revenue_v2.loc[domestic_revenue_v2.duplicated(subset=["name"]), "name"].unique()

In [32]:
domestic_revenue_v2 = domestic_revenue_v2.rename(columns={'gross': 'domestic_revenue_usd',
                                                         'name': 'name_domestic'})

In [33]:
domestic_revenue_v2

,budget,company,director,genre,domestic_revenue_usd,name_domestic,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes
2,15000000,Paramount Pictures,Tony Scott,Action,179800601,Top gun,PG,5/16/86,110,6.9,Tom Cruise,236909,Jim Cash
3,18500000,Twentieth Century Fox Film Corporation,James Cameron,Action,85160248,Aliens,R,7/18/86,137,8.4,Sigourney Weaver,540152,James Cameron
4,9000000,Walt Disney Pictures,Randal Kleiser,Adventure,18564613,Flight of the navigator,PG,8/1/86,90,6.9,Joey Cramer,36636,Mark H. Baker
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6815,0,Fox Searchlight Pictures,Mandie Fletcher,Comedy,4750497,Absolutely fabulous: the movie,R,7/22/16,91,5.4,Jennifer Saunders,9161,Jennifer Saunders
6816,0,Siempre Viva Productions,Paul Duddridge,Drama,28368,Mothers and daughters,PG-13,5/6/16,90,4.9,Selma Blair,1959,Paige Cameron
6817,3500000,Warner Bros. Animation,Sam Liu,Animation,3775000,Batman: the killing joke,R,7/25/16,76,6.5,Kevin Conroy,36333,Brian Azzarello
6818,0,Borderline Presents,Nicolas Pesce,Drama,25981,The eyes of my mother,R,12/2/16,76,6.2,Kika Magalh„es,6947,Nicolas Pesce


In [34]:
domestic_revenue_v2.to_csv('Resources/Domestic_v3.csv')

### Use pandas to load csv converted DataFrame into database

### Mac (Irina) Connection

In [35]:
# # https://splinter.readthedocs.io/en/latest/drivers/chrome.html
# !which chromedriver

In [36]:
# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

### Windows (Clay) Connection

In [37]:
# executable_path = {'executable_path': 'C:/Users/Clay/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

### Connect to local database

In [38]:
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [39]:
engine.table_names()

['international_revenue', 'best_200_directors']

In [40]:
# Create international revenue table in the database
international_v2.to_sql(name='international_revenue', con=engine, if_exists='append', index=False)

In [41]:
# Create domesic revenue table in the database
domestic_revenue_v2.to_sql(name='domestic_revenue', con=engine, if_exists='append', index=False)

In [42]:
engine.table_names()

['international_revenue', 'best_200_directors', 'domestic_revenue']

### Confirm data has been added by querying the international table

In [43]:
pd.read_sql_query("select * from international_revenue", con=engine).head()

,name_international,year,international_revenue_usd
0,Avengers: endgame,2019,2797800564
1,Avatar,2009,2790439000
2,Titanic,1997,2195169138
3,Star wars: episode vii - the force awakens,2015,2068223624
4,Avengers: infinity war,2018,2048359754


In [44]:
pd.read_sql_query('select * from domestic_revenue', con=engine).head()

,budget,company,director,genre,domestic_revenue_usd,name_domestic,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes
2,15000000,Paramount Pictures,Tony Scott,Action,179800601,Top gun,PG,5/16/86,110,6.9,Tom Cruise,236909,Jim Cash
3,18500000,Twentieth Century Fox Film Corporation,James Cameron,Action,85160248,Aliens,R,7/18/86,137,8.4,Sigourney Weaver,540152,James Cameron
4,9000000,Walt Disney Pictures,Randal Kleiser,Adventure,18564613,Flight of the navigator,PG,8/1/86,90,6.9,Joey Cramer,36636,Mark H. Baker


### Best movie directors of all times

In [99]:
best_director_url  = 'https://www.imdb.com/list/ls056848274/'
best_director_url2 = 'https://www.imdb.com/list/ls056848274/?sort=list_order,asc&mode=detail&page=2'

In [100]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [101]:
browser.visit(best_director_url)
# time.sleep(5)

In [57]:
results = browser.find_by_css("h3")
results

In [58]:
list_original = []
for i in results:
    print(i.text)
    list_original.append(i.text)

1. Steven Spielberg
2. Martin Scorsese
3. Alfred Hitchcock
4. Stanley Kubrick
5. Quentin Tarantino
6. Orson Welles
7. Francis Ford Coppola
8. Ridley Scott
9. Akira Kurosawa
10. Joel Coen
11. Ethan Coen
12. John Ford
13. Sergio Leone
14. Woody Allen
15. Billy Wilder
16. Clint Eastwood
17. Frank Capra
18. Charles Chaplin
19. Christopher Nolan
20. Roman Polanski
21. Ingmar Bergman
22. David Lynch
23. Fritz Lang
24. David Fincher
25. Peter Jackson
26. Federico Fellini
27. Tim Burton
28. James Cameron
29. Sidney Lumet
30. Brian De Palma
31. François Truffaut
32. Terrence Malick
33. David Cronenberg
34. William Wyler
35. Robert Zemeckis
36. Ang Lee
37. Howard Hawks
38. George Lucas
39. Mel Brooks
40. Terry Gilliam
41. Milos Forman
42. Steven Soderbergh
43. John Huston
44. Werner Herzog
45. Oliver Stone
46. David Lean
47. Danny Boyle
48. Rob Reiner
49. Buster Keaton
50. Michael Curtiz
51. Sam Raimi
52. Michael Mann
53. Robert Altman
54. Sam Peckinpah
55. Peter Weir
56. Spike Lee
57. Jean Reno

In [59]:
directors_list_100 = list_original[:100]
directors_list_100

['1. Steven Spielberg',
 '2. Martin Scorsese',
 '3. Alfred Hitchcock',
 '4. Stanley Kubrick',
 '5. Quentin Tarantino',
 '6. Orson Welles',
 '7. Francis Ford Coppola',
 '8. Ridley Scott',
 '9. Akira Kurosawa',
 '10. Joel Coen',
 '11. Ethan Coen',
 '12. John Ford',
 '13. Sergio Leone',
 '14. Woody Allen',
 '15. Billy Wilder',
 '16. Clint Eastwood',
 '17. Frank Capra',
 '18. Charles Chaplin',
 '19. Christopher Nolan',
 '20. Roman Polanski',
 '21. Ingmar Bergman',
 '22. David Lynch',
 '23. Fritz Lang',
 '24. David Fincher',
 '25. Peter Jackson',
 '26. Federico Fellini',
 '27. Tim Burton',
 '28. James Cameron',
 '29. Sidney Lumet',
 '30. Brian De Palma',
 '31. François Truffaut',
 '32. Terrence Malick',
 '33. David Cronenberg',
 '34. William Wyler',
 '35. Robert Zemeckis',
 '36. Ang Lee',
 '37. Howard Hawks',
 '38. George Lucas',
 '39. Mel Brooks',
 '40. Terry Gilliam',
 '41. Milos Forman',
 '42. Steven Soderbergh',
 '43. John Huston',
 '44. Werner Herzog',
 '45. Oliver Stone',
 '46. David 

In [60]:
clean_list1 = []
for i in directors_list_100:
    director = i.split('. ')[-1]
    print(director)
    clean_list1.append(director)

Steven Spielberg
Martin Scorsese
Alfred Hitchcock
Stanley Kubrick
Quentin Tarantino
Orson Welles
Francis Ford Coppola
Ridley Scott
Akira Kurosawa
Joel Coen
Ethan Coen
John Ford
Sergio Leone
Woody Allen
Billy Wilder
Clint Eastwood
Frank Capra
Charles Chaplin
Christopher Nolan
Roman Polanski
Ingmar Bergman
David Lynch
Fritz Lang
David Fincher
Peter Jackson
Federico Fellini
Tim Burton
James Cameron
Sidney Lumet
Brian De Palma
François Truffaut
Terrence Malick
David Cronenberg
William Wyler
Robert Zemeckis
Ang Lee
Howard Hawks
George Lucas
Mel Brooks
Terry Gilliam
Milos Forman
Steven Soderbergh
John Huston
Werner Herzog
Oliver Stone
David Lean
Danny Boyle
Rob Reiner
Buster Keaton
Michael Curtiz
Sam Raimi
Michael Mann
Robert Altman
Sam Peckinpah
Peter Weir
Spike Lee
Jean Renoir
Andrei Tarkovsky
Elia Kazan
John Cassavetes
Vincente Minnelli
Hayao Miyazaki
Frank Darabont
Kathryn Bigelow
George Cukor
Ron Howard
John Carpenter
Guy Ritchie
Tony Scott
Gus Van Sant
Preston Sturges
Eisenstein
Russel

In [61]:
clean_list1

['Steven Spielberg',
 'Martin Scorsese',
 'Alfred Hitchcock',
 'Stanley Kubrick',
 'Quentin Tarantino',
 'Orson Welles',
 'Francis Ford Coppola',
 'Ridley Scott',
 'Akira Kurosawa',
 'Joel Coen',
 'Ethan Coen',
 'John Ford',
 'Sergio Leone',
 'Woody Allen',
 'Billy Wilder',
 'Clint Eastwood',
 'Frank Capra',
 'Charles Chaplin',
 'Christopher Nolan',
 'Roman Polanski',
 'Ingmar Bergman',
 'David Lynch',
 'Fritz Lang',
 'David Fincher',
 'Peter Jackson',
 'Federico Fellini',
 'Tim Burton',
 'James Cameron',
 'Sidney Lumet',
 'Brian De Palma',
 'François Truffaut',
 'Terrence Malick',
 'David Cronenberg',
 'William Wyler',
 'Robert Zemeckis',
 'Ang Lee',
 'Howard Hawks',
 'George Lucas',
 'Mel Brooks',
 'Terry Gilliam',
 'Milos Forman',
 'Steven Soderbergh',
 'John Huston',
 'Werner Herzog',
 'Oliver Stone',
 'David Lean',
 'Danny Boyle',
 'Rob Reiner',
 'Buster Keaton',
 'Michael Curtiz',
 'Sam Raimi',
 'Michael Mann',
 'Robert Altman',
 'Sam Peckinpah',
 'Peter Weir',
 'Spike Lee',
 'Je

In [62]:
browser.visit(best_director_url2)
# time.sleep(5)

In [63]:
results2 = browser.find_by_css("h3")
results2

In [64]:
original_list2 = []
for i in results2:
#     print(i.text)
    original_list2.append(i.text)

In [65]:
directors_list2 = original_list2[:100]
directors_list2

['101. Jean-Luc Godard',
 '102. Tom Hooper',
 '103. William Friedkin',
 '104. Antoine Fuqua',
 '105. Sydney Pollack',
 '106. Bernardo Bertolucci',
 '107. John Lasseter',
 '108. David Ayer',
 '109. J.J. Abrams',
 '110. Mike Nichols',
 '111. Lars von Trier',
 '112. John Milius',
 '113. John McTiernan',
 '114. Costa-Gavras',
 '115. Cecil B. DeMille',
 '116. Darren Aronofsky',
 '117. Luc Besson',
 '118. Doug Liman',
 '119. Bryan Singer',
 '120. Richard Linklater',
 '121. Anthony Minghella',
 '122. Curtis Hanson',
 '123. Michael Haneke',
 '124. D.W. Griffith',
 '125. Alan Parker',
 '126. Edgar Wright',
 '127. Pedro Almodóvar',
 '128. Barry Levinson',
 '129. Michelangelo Antonioni',
 '130. Cameron Crowe',
 '131. Jim Jarmusch',
 '132. Vittorio De Sica',
 '133. Spike Jonze',
 '134. Harold Ramis',
 '135. Michael Moore',
 '136. Shane Meadows',
 '137. Ken Loach',
 '138. Bobby Farrelly',
 '139. Jacques Audiard',
 '140. Gore Verbinski',
 '141. Jay Roach',
 '142. Jim Sheridan',
 '143. John Landis',


In [66]:
clean_list2 = []
for i in directors_list2:
    director = i.split('. ')[-1]
    clean_list2.append(director)

In [67]:
clean_list2

['Jean-Luc Godard',
 'Tom Hooper',
 'William Friedkin',
 'Antoine Fuqua',
 'Sydney Pollack',
 'Bernardo Bertolucci',
 'John Lasseter',
 'David Ayer',
 'Abrams',
 'Mike Nichols',
 'Lars von Trier',
 'John Milius',
 'John McTiernan',
 'Costa-Gavras',
 'DeMille',
 'Darren Aronofsky',
 'Luc Besson',
 'Doug Liman',
 'Bryan Singer',
 'Richard Linklater',
 'Anthony Minghella',
 'Curtis Hanson',
 'Michael Haneke',
 'Griffith',
 'Alan Parker',
 'Edgar Wright',
 'Pedro Almodóvar',
 'Barry Levinson',
 'Michelangelo Antonioni',
 'Cameron Crowe',
 'Jim Jarmusch',
 'Vittorio De Sica',
 'Spike Jonze',
 'Harold Ramis',
 'Michael Moore',
 'Shane Meadows',
 'Ken Loach',
 'Bobby Farrelly',
 'Jacques Audiard',
 'Gore Verbinski',
 'Jay Roach',
 'Jim Sheridan',
 'John Landis',
 'Richard Donner',
 'Jason Reitman',
 'Robert Redford',
 'Andrew Dominik',
 'Joss Whedon',
 'George Miller',
 'David Zucker',
 'Romero',
 'Mike Leigh',
 'Russell',
 'Neil Jordan',
 'Wilfred Jackson',
 'Pakula',
 'George Roy Hill',
 'M

In [68]:
directors = clean_list1 + clean_list2
directors

['Steven Spielberg',
 'Martin Scorsese',
 'Alfred Hitchcock',
 'Stanley Kubrick',
 'Quentin Tarantino',
 'Orson Welles',
 'Francis Ford Coppola',
 'Ridley Scott',
 'Akira Kurosawa',
 'Joel Coen',
 'Ethan Coen',
 'John Ford',
 'Sergio Leone',
 'Woody Allen',
 'Billy Wilder',
 'Clint Eastwood',
 'Frank Capra',
 'Charles Chaplin',
 'Christopher Nolan',
 'Roman Polanski',
 'Ingmar Bergman',
 'David Lynch',
 'Fritz Lang',
 'David Fincher',
 'Peter Jackson',
 'Federico Fellini',
 'Tim Burton',
 'James Cameron',
 'Sidney Lumet',
 'Brian De Palma',
 'François Truffaut',
 'Terrence Malick',
 'David Cronenberg',
 'William Wyler',
 'Robert Zemeckis',
 'Ang Lee',
 'Howard Hawks',
 'George Lucas',
 'Mel Brooks',
 'Terry Gilliam',
 'Milos Forman',
 'Steven Soderbergh',
 'John Huston',
 'Werner Herzog',
 'Oliver Stone',
 'David Lean',
 'Danny Boyle',
 'Rob Reiner',
 'Buster Keaton',
 'Michael Curtiz',
 'Sam Raimi',
 'Michael Mann',
 'Robert Altman',
 'Sam Peckinpah',
 'Peter Weir',
 'Spike Lee',
 'Je

In [84]:
len(directors)

200

In [91]:
directors_df = pd.DataFrame(directors)

In [92]:
len(directors_df)

200

In [86]:
# # Shift the index to start from 1 as the list is ranked
# directors_df = directors_df.shift()[1:]

In [93]:
# Rename the column
directors_df.rename(columns={0: 'directors'}, inplace=True)

In [94]:
directors_df.reset_index()

,index,directors
0,0,Steven Spielberg
1,1,Martin Scorsese
2,2,Alfred Hitchcock
3,3,Stanley Kubrick
4,4,Quentin Tarantino
...,...,...
195,195,Theodoros Angelopoulos
196,196,James Whale
197,197,Terence Fisher
198,198,Chuck Jones


In [95]:
# Save directors to csv
directors_df.to_csv('Resources/Best_directors_200.csv')

In [96]:
len(directors_df)

200

#### Push Directors table to sql

In [97]:
directors_df.to_sql(name='best_200_directors', con=engine, if_exists='append', index=False)

In [98]:
pd.read_sql_query('select * from best_200_directors', con=engine).head()

,directors
0,Steven Spielberg
1,Martin Scorsese
2,Alfred Hitchcock
3,Stanley Kubrick
4,Quentin Tarantino


In [99]:
engine.table_names()

['international_v2', 'domestic_revenue_v2', 'best_200_directors']

#### Get director's name with the featured movie

In [118]:
results = browser.find_by_css("span.ghost")

In [119]:
results

In [120]:
len(results)

103

In [124]:
for i in results:
    print(i.childNodes)

AttributeError: 'WebDriverElement' object has no attribute 'childNodes'

In [58]:
list_original = []
for i in results:
    print(i.text)
    list_original.append(i.text)

1. Steven Spielberg
2. Martin Scorsese
3. Alfred Hitchcock
4. Stanley Kubrick
5. Quentin Tarantino
6. Orson Welles
7. Francis Ford Coppola
8. Ridley Scott
9. Akira Kurosawa
10. Joel Coen
11. Ethan Coen
12. John Ford
13. Sergio Leone
14. Woody Allen
15. Billy Wilder
16. Clint Eastwood
17. Frank Capra
18. Charles Chaplin
19. Christopher Nolan
20. Roman Polanski
21. Ingmar Bergman
22. David Lynch
23. Fritz Lang
24. David Fincher
25. Peter Jackson
26. Federico Fellini
27. Tim Burton
28. James Cameron
29. Sidney Lumet
30. Brian De Palma
31. François Truffaut
32. Terrence Malick
33. David Cronenberg
34. William Wyler
35. Robert Zemeckis
36. Ang Lee
37. Howard Hawks
38. George Lucas
39. Mel Brooks
40. Terry Gilliam
41. Milos Forman
42. Steven Soderbergh
43. John Huston
44. Werner Herzog
45. Oliver Stone
46. David Lean
47. Danny Boyle
48. Rob Reiner
49. Buster Keaton
50. Michael Curtiz
51. Sam Raimi
52. Michael Mann
53. Robert Altman
54. Sam Peckinpah
55. Peter Weir
56. Spike Lee
57. Jean Reno